In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/gender_submission.csv
/kaggle/input/titanic/test.csv


In [2]:
import pandas as pd
df=pd.read_csv('../input/titanic/train.csv')
df.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
x=df.iloc[:,[0,2,3,4,5,6,7,8,9]]
x.head(5)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500


In [4]:
y=df.iloc[:,1]
y.head(5)

0    0
1    1
2    1
3    1
4    0
Name: Survived, dtype: int64

Number of unique values are seen, especially for Pclass column.

In [5]:
r,c=x.shape
#print(r,' ',c)
x.nunique()

PassengerId    891
Pclass           3
Name           891
Sex              2
Age             88
SibSp            7
Parch            7
Ticket         681
Fare           248
dtype: int64

Number of Missing values are seen below.

In [6]:
x.isnull().sum(axis=0)

PassengerId      0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
dtype: int64

1st trying not filling missing values by anything.

Various method can be tried-

1.Multiple Linear Regression 
    
    a.with all columns
    b.Dropping names and ticket because it seems it my not affect the result.
2.Take only age and gender.

    a.Give preferrence for age below 12 and greater than 60. And if female then more chances of alive.
    b.Put these 2 columns directly into algo.
    
3.Logistic Regression with all columns

In [7]:
import matplotlib.pyplot as plt

In [8]:
df.groupby('Survived').size()

Survived
0    549
1    342
dtype: int64

In [9]:
counts=df['Survived'].value_counts()

In [10]:
counts.index
counts = df['Survived'].value_counts()
dead=counts[0]
alive= counts[1]
print(dead,alive)

549 342


Male and female have to be encoded. Same with names

In [11]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder = LabelEncoder()
x.iloc[:, 3] = labelencoder.fit_transform(x.iloc[:, 3])


/opt/conda/lib/python3.6/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [12]:
x.drop(['Name'], axis = 1, inplace = True) 
x.drop(['Ticket'], axis = 1, inplace = True) 

/opt/conda/lib/python3.6/site-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [13]:
x

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare
0,1,3,1,22.0,1,0,7.2500
1,2,1,0,38.0,1,0,71.2833
2,3,3,0,26.0,0,0,7.9250
3,4,1,0,35.0,1,0,53.1000
4,5,3,1,35.0,0,0,8.0500
...,...,...,...,...,...,...,...
886,887,2,1,27.0,0,0,13.0000
887,888,1,0,19.0,0,0,30.0000
888,889,3,0,NaN,1,2,23.4500
889,890,1,1,26.0,0,0,30.0000


In [14]:
# unable to apply onehotencoder
# one_age=OneHotEncoder()
# x=one_age.fit_transform(x).toarray()

In [15]:
x

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare
0,1,3,1,22.0,1,0,7.2500
1,2,1,0,38.0,1,0,71.2833
2,3,3,0,26.0,0,0,7.9250
3,4,1,0,35.0,1,0,53.1000
4,5,3,1,35.0,0,0,8.0500
...,...,...,...,...,...,...,...
886,887,2,1,27.0,0,0,13.0000
887,888,1,0,19.0,0,0,30.0000
888,889,3,0,NaN,1,2,23.4500
889,890,1,1,26.0,0,0,30.0000


### Multiple Linear Regression

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)


In [17]:
import statsmodels.api as sm
x=np.append(arr=np.ones((891,1)),values=x,axis=1)


In [18]:
x.shape

(891, 8)

In [19]:
# 

In [20]:
xopt=x[:,[0,1,2,3,5,6,7]]
regosl=sm.OLS(endog=y,exog=xopt).fit()
regosl.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               Survived   R-squared:                       0.375
Model:                            OLS   Adj. R-squared:                  0.370
Method:                 Least Squares   F-statistic:                     88.22
Date:                Fri, 27 Dec 2019   Prob (F-statistic):           1.18e-86
Time:                        05:07:47   Log-Likelihood:                -412.92
No. Observations:                 891   AIC:                             839.8
Df Residuals:                     884   BIC:                             873.4
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.0454      0.058     17.874      0.000       0.931       1.160
x1          7.501e-06   5.05e-05      0.149      0.882   -9.16e-05       0.000
x2            -0.1376      0.019     -7.208      0.000      -0.175      -0.100
x3            -0.5262      0.028    -18.582      0.000      -0.582      -0.471
x4            -0.0326      0.013     -2.485      0.013      -0.058      -0.007
x5            -0.0126      0.018     -0.687      0.492      -0.049       0.023
x6             0.0005      0.000      1.472      0.141      -0.000       0.001
==============================================================================
Omnibus:                       44.950   Durbin-Watson:                   1.926
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               50.776
Skew:                           0.583   Prob(JB):                     9.42e-12
Kurtosis:                       3.089   Cond. No.                     2.43e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.43e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [21]:
xopt=x[:,[0,2,3,5,6,7]]
regosl=sm.OLS(endog=y,exog=xopt).fit()
regosl.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               Survived   R-squared:                       0.374
Model:                            OLS   Adj. R-squared:                  0.371
Method:                 Least Squares   F-statistic:                     106.0
Date:                Fri, 27 Dec 2019   Prob (F-statistic):           1.10e-87
Time:                        05:07:47   Log-Likelihood:                -412.93
No. Observations:                 891   AIC:                             837.9
Df Residuals:                     885   BIC:                             866.6
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.0488      0.054     19.510      0.000       0.943       1.154
x1            -0.1377      0.019     -7.219      0.000      -0.175      -0.100
x2            -0.5260      0.028    -18.607      0.000      -0.582      -0.471
x3            -0.0327      0.013     -2.499      0.013      -0.058      -0.007
x4            -0.0125      0.018     -0.683      0.495      -0.049       0.023
x5             0.0005      0.000      1.473      0.141      -0.000       0.001
==============================================================================
Omnibus:                       44.887   Durbin-Watson:                   1.926
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               50.695
Skew:                           0.583   Prob(JB):                     9.81e-12
Kurtosis:                       3.089   Cond. No.                         261.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [22]:
xopt=x[:,[0,2,3,5,7]]
regosl=sm.OLS(endog=y,exog=xopt).fit()
regosl.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               Survived   R-squared:                       0.374
Model:                            OLS   Adj. R-squared:                  0.371
Method:                 Least Squares   F-statistic:                     132.4
Date:                Fri, 27 Dec 2019   Prob (F-statistic):           1.13e-88
Time:                        05:07:47   Log-Likelihood:                -413.17
No. Observations:                 891   AIC:                             836.3
Df Residuals:                     886   BIC:                             860.3
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.0477      0.054     19.505      0.000       0.942       1.153
x1            -0.1392      0.019     -7.341      0.000      -0.176      -0.102
x2            -0.5221      0.028    -18.868      0.000      -0.576      -0.468
x3            -0.0360      0.012     -2.947      0.003      -0.060      -0.012
x4             0.0004      0.000      1.376      0.169      -0.000       0.001
==============================================================================
Omnibus:                       43.112   Durbin-Watson:                   1.927
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               48.432
Skew:                           0.570   Prob(JB):                     3.04e-11
Kurtosis:                       3.083   Cond. No.                         261.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [23]:
xopt=x[:,[0,2,3,5]]
regosl=sm.OLS(endog=y,exog=xopt).fit()
regosl.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               Survived   R-squared:                       0.373
Model:                            OLS   Adj. R-squared:                  0.371
Method:                 Least Squares   F-statistic:                     175.8
Date:                Fri, 27 Dec 2019   Prob (F-statistic):           2.01e-89
Time:                        05:07:47   Log-Likelihood:                -414.12
No. Observations:                 891   AIC:                             836.2
Df Residuals:                     887   BIC:                             855.4
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.0964      0.040     27.119      0.000       1.017       1.176
x1            -0.1538      0.016     -9.803      0.000      -0.185      -0.123
x2            -0.5261      0.028    -19.110      0.000      -0.580      -0.472
x3            -0.0320      0.012     -2.699      0.007      -0.055      -0.009
==============================================================================
Omnibus:                       42.380   Durbin-Watson:                   1.927
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               47.482
Skew:                           0.564   Prob(JB):                     4.89e-11
Kurtosis:                       3.088   Cond. No.                         9.35
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [24]:
X_train=pd.DataFrame(X_train).to_numpy()
X_test=pd.DataFrame(X_test).to_numpy()

In [25]:
X_trainNew=X_train[:,[0,2,5]]

In [26]:
type(X_test)

numpy.ndarray

In [27]:
X_testNew=X_test[:,[0,2,5]]

In [28]:
X_trainNew = X_trainNew.astype('float64') 
X_testNew = X_testNew.astype('float64') 


In [29]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(X_trainNew, y_train)
ypred1=regressor.predict(X_testNew)

In [30]:
y_test

495    0
648    0
278    0
31     1
255    1
      ..
780    1
837    0
215    1
833    0
372    0
Name: Survived, Length: 179, dtype: int64

In [31]:
# ypred1=pd.Series(ypred1)

In [32]:
k=0
for i in ypred1:
    if i>=0.5:
        ypred1[k]=1
    else:
        ypred1[k]=0
    k=k+1

In [33]:
ypred1=pd.Series(ypred1)

In [34]:
y_test

495    0
648    0
278    0
31     1
255    1
      ..
780    1
837    0
215    1
833    0
372    0
Name: Survived, Length: 179, dtype: int64

In [35]:
ypred1 = ypred1.astype('Int64')
ypred1

0      0
1      0
2      0
3      1
4      1
      ..
174    1
175    0
176    1
177    0
178    0
Length: 179, dtype: Int64

In [36]:
type(ypred1)

pandas.core.series.Series

In [37]:
y_test.shape

(179,)

In [38]:
pred_res1=0
pred_res0=0
for i in ypred1:
    if i>=0.5:
        pred_res1=pred_res1+1
    else:
        pred_res0=pred_res0+1

print(pred_res1,pred_res0)

67 112


In [39]:
acc_res1=0
acc_res0=0
for i in y_test:
    if i>=0.5:
        acc_res1=acc_res1+1
    else:
        acc_res0=acc_res0+1

print(acc_res1,acc_res0)

69 110


In [40]:
dftest=pd.read_csv('../input/titanic/test.csv')
dftest.head(5)
x1=dftest.iloc[:,[0,1,2,3,4,5,6,7,8,9]]

In [41]:
x1

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN
...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN


In [42]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder = LabelEncoder()
x1.iloc[:, 3] = labelencoder.fit_transform(x1.iloc[:, 3])


/opt/conda/lib/python3.6/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [43]:
x1.drop(['Name'], axis = 1, inplace = True) 
x1.drop(['Ticket'], axis = 1, inplace = True) 

/opt/conda/lib/python3.6/site-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [44]:
x1

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin
0,892,3,1,34.5,0,0,7.8292,NaN
1,893,3,0,47.0,1,0,7.0000,NaN
2,894,2,1,62.0,0,0,9.6875,NaN
3,895,3,1,27.0,0,0,8.6625,NaN
4,896,3,0,22.0,1,1,12.2875,NaN
...,...,...,...,...,...,...,...,...
413,1305,3,1,NaN,0,0,8.0500,NaN
414,1306,1,0,39.0,0,0,108.9000,C105
415,1307,3,1,38.5,0,0,7.2500,NaN
416,1308,3,1,NaN,0,0,8.0500,NaN


In [45]:
type(x1)
x1=pd.DataFrame(x1).to_numpy()

In [46]:
X_testNew1=x1[:,[0,2,5]]

In [47]:
# X_testNew1=pd.DataFrame(X_testNew1)
# X_testNew1.isnull().sum(axis=0)

In [48]:
ypred2=regressor.predict(X_testNew1)

In [49]:
k=0
for i in ypred2:
    if i>=0.5:
        ypred2[k]=1
    else:
        ypred2[k]=0
    k=k+1

In [50]:
ypred2 = ypred2.astype(int)  #Converted from float to int because answer is reqd in int. Otherwise score = 0.0 
ypred2


array([0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [51]:
p1=dftest.iloc[:,[0]]
p2= pd.DataFrame({'Sur': ypred2})
p1,p2
# p1.append(p2) When appended, I get all values as nan....don't know why

(     PassengerId
 0            892
 1            893
 2            894
 3            895
 4            896
 ..           ...
 413         1305
 414         1306
 415         1307
 416         1308
 417         1309
 
 [418 rows x 1 columns],      Sur
 0      0
 1      1
 2      0
 3      0
 4      1
 ..   ...
 413    0
 414    1
 415    0
 416    0
 417    0
 
 [418 rows x 1 columns])

In [52]:
p1.isnull().sum(axis=0)

PassengerId    0
dtype: int64

In [53]:
PassengerId=dftest['PassengerId']
PassengerId

0       892
1       893
2       894
3       895
4       896
       ... 
413    1305
414    1306
415    1307
416    1308
417    1309
Name: PassengerId, Length: 418, dtype: int64

In [54]:
submission=pd.DataFrame({'PassengerId':PassengerId})
submission['Survived']=ypred2

In [55]:
submission.head()


,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [56]:
submission.to_csv('TitanicResult2.csv',index=False)

In [57]:
ypred1 = ypred1.astype(int)
ypred1

0      0
1      0
2      0
3      1
4      1
      ..
174    1
175    0
176    1
177    0
178    0
Length: 179, dtype: int64

In [58]:
from sklearn.metrics import accuracy_score 

In [59]:
accuracy_score(y_test,ypred1)

0.7877094972067039